In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os
from IPython.core.interactiveshell import InteractiveShell

sys.path.insert(-1, os.getcwd())
InteractiveShell.ast_node_interactivity = "all"
os.environ['THEANO_FLAGS'] = "device=cpu"

In [ ]:
import numpy as np
import tensorflow as tf
import theano
import theano.tensor as T

from ctc import ctc_loss as my_ctc_loss

In [ ]:
batch_size = 16
max_labsize = 20
voca_size = 20  # excluding blank
max_seqsize = 100
blank = -1

labsize = np.random.randint(
    1, max_labsize + 1, size=(batch_size,), dtype=np.int32)
labsize[0] = max_labsize
labsize[1] = 1
labsize[2] = max_labsize
labsize[3] = max_labsize

labels = np.random.randint(
    0, voca_size,
    size=(batch_size, max_labsize), dtype=np.int32)
for b in range(batch_size):
    labels[b, labsize[b]:] = blank

seqsize = np.array([
    np.random.randint(labsize[i] + 1, max_seqsize + 1)
    for i in range(batch_size)], dtype=np.int32)

linout = np.random.randn(
    max_seqsize, batch_size, voca_size + 1).astype(np.float32)

In [ ]:
th_linout_var = T.tensor3()
th_seqsize_var = T.ivector()
th_labels_var = T.imatrix()
th_labsize_var = T.ivector()
th_loss = my_ctc_loss(th_linout_var, th_seqsize_var, th_labels_var, th_labsize_var)

def dense_to_sparse(x):
    idx = tf.where(tf.greater_equal(x, 0))
    return tf.SparseTensor(idx, tf.gather_nd(x, idx), tf.shape(x, out_type=tf.int64))

tf_linout_var = tf.placeholder(tf.float32, shape=[max_seqsize, batch_size, voca_size + 1])
tf_seqsize_var = tf.placeholder(tf.int32, shape=[batch_size])
tf_labels_var = tf.placeholder(tf.int32, shape=[batch_size, max_labsize])

tf_loss = tf.nn.ctc_loss(
    dense_to_sparse(tf_labels_var), tf_linout_var,
    sequence_length=tf_seqsize_var,
    time_major=True)

In [ ]:
with tf.Session() as sess:
    tf_result = sess.run(
        tf_loss, {
            tf_linout_var: linout,
            tf_seqsize_var: seqsize,
            tf_labels_var: labels
        })
    
    th_results = th_loss.eval({
        th_linout_var: linout,
        th_seqsize_var: seqsize,
        th_labels_var: labels,
        th_labsize_var: labsize
    })
    
    print(np.abs(tf_result - th_results) / tf_result)

In [ ]:
tf_g = tf.gradients(xs=tf_linout_var, ys=tf.reduce_sum(tf_loss))[0]

with tf.Session() as sess:
    tf_grad = sess.run(
        tf_g, {
            tf_linout_var: linout,
            tf_seqsize_var: seqsize,
            tf_labels_var: labels
        })
    
    th_grad = theano.grad(th_loss.sum(), wrt=th_linout_var).eval({
        th_linout_var: linout,
        th_seqsize_var: seqsize,
        th_labels_var: labels,
        th_labsize_var: labsize
    })
    
    print(np.abs(tf_grad - th_grad) / (tf_grad + .000001))

In [ ]:
th_grad[:, 1, :]

In [ ]:
tf_grad[:, 1, :]